In [2]:
import os
import requests
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep


from locations import *
from parsing import *

# LOCATIONS SCRAPING

In [3]:
# Base

base_url = 'https://www.pisos.com/'

venta_url = f'{base_url}venta/'

In [4]:
# Sacamos todas las paginas filtradas por provincias

response = requests.get(base_url)

soup = BeautifulSoup(response.text, "html.parser")

provincias_str = soup.find('div', class_ = 'home-container').find('div', class_ = 'selectBox').find_all('ul')[-1].text

provincias = [x.lower() for x in provincias_str.strip().split('\n')]

endpoints = replaceWithUnderscore(provincias)

endpoints = [venta_url + endpoint for endpoint in endpoints]

with open('locations.txt', 'w') as file:
    file.write('\n'.join(endpoints))

# SINGLE PAGE SCRAPING

In [5]:
def scrape(url):

    browser = webdriver.Chrome()
    browser.get(url)
    browser.maximize_window()

    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    )

    element.click() # Accept cookies

    for i in range(30):

        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, f'(//*[@id]/div[2]/div[1]/div[2]/a)[{i + 1}]'))
        )

        element.click() # Click on the first post

        # <SCROLLING>
        while True:
            is_at_bottom = browser.execute_script("return window.scrollY + window.innerHeight >= document.body.scrollHeight")
            browser.execute_script("window.scroll({ top: document.body.scrollHeight, behavior: 'smooth' });")
            if is_at_bottom:
                break
            sleep(0.2)
        # </SCROLLING>
        
        html_content = browser.page_source
        soft_url = url.replace('https://www.pisos.com/venta/', '')
        soft_url = soft_url.replace('/', '_')
        soft_url = soft_url.replace('-', '_')

        file_path = f'{os.getcwd()}/html_content/{soft_url}_{i}.html'
        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(html_content)
        except Exception as e:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(repr(e))

        go_back_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="GoBackLink"]'))
        )

        go_back_element.click() # Click on back arrow

    browser.quit()

    

# TESTING

In [19]:
def scanRegions(url):
    response = requests.get(url)

    print(f'URL: {url} | STATUS {response.status_code}')

    soup = BeautifulSoup(response.text, 'html.parser')

    items = soup.select('div.zoneList a.item:not(.item-subitem)')

    endpoints = {}
    for item in items:
        endpoint = item['href']
        n_results = item.find('span', class_ = 'total').text

        # <INT>
        if len(n_results) != 0:
            try:
                n_results = n_results[1:-1]
                n_results = ''.join(n_results.split('.'))
                n_results = int(n_results)
            except:
                print(f'FAIL CASTING TO INTEGER {endpoint}')
        else:
            print(f'n_results EMPTY {endpoint}')
        #</INT>

        # <RECURSSION>
        if n_results > 3000:
            endpoints[endpoint] = scanRegions(base_url[:-1] + endpoint)
        # </RECURSSION>

        else: endpoints[endpoint] = n_results

    return endpoints

In [20]:
def parseRegions(endpoints):

    def extract(endpoints):
        array = []
        for key, value in endpoints.items():
            if isinstance(value, int):
                array.append(key)
                continue

            data = extract(value)
            array.extend(data)

        return array
    
    endpoints = extract(endpoints)

    array = []
    
    for endpoint in endpoints:
        data = endpoint
        if '/venta/pisos-' in endpoint:
            data = endpoint.replace('/venta/pisos-', '/viviendas/')
        array.append(data)

    return array


In [24]:
URL = 'https://www.pisos.com/viviendas/barcelona/'

urls = scanRegions(URL)
urls = parseRegions(urls)
urls = [base_url + x for x in urls]
urls = [x.replace('d//vivienas/', '/venta/pisos-') for x in urls]

URL: https://www.pisos.com/viviendas/barcelona/ | STATUS 200
[<a class="item" data-highlight="true" href="/viviendas/alt_penedes/"><span class="text">Alt Penedès</span> <span class="total">(632)</span></a>, <a class="item" data-highlight="true" href="/viviendas/anoia/"><span class="text">Anoia</span> <span class="total">(586)</span></a>, <a class="item" data-highlight="true" href="/viviendas/bages/"><span class="text">Bages</span> <span class="total">(1.071)</span></a>, <a class="item" data-highlight="true" href="/viviendas/baix_llobregat/"><span class="text">Baix Llobregat</span> <span class="total">(3.043)</span></a>, <a class="item" data-highlight="true" href="/viviendas/barcelones/"><span class="text">Barcelonès</span> <span class="total">(7.741)</span></a>, <a class="item" data-highlight="true" href="/viviendas/bergueda/"><span class="text">Berguedà</span> <span class="total">(144)</span></a>, <a class="item" data-highlight="true" href="/viviendas/garraf/"><span class="text">Garra

In [25]:
print(urls)

['https://www.pisos.com//viviendas/alt_penedes/', 'https://www.pisos.com//viviendas/anoia/', 'https://www.pisos.com//viviendas/bages/', 'https://www.pisos.com//viviendas/baix_llobregat_nord/', 'https://www.pisos.com//viviendas/baix_llobregat_sud_est/', 'https://www.pisos.com//viviendas/baix_llobregat_sud_oest/', 'https://www.pisos.com//viviendas/badalona/', 'https://www.pisos.com//viviendas/ciutat_vella/', 'https://www.pisos.com//viviendas/barcelona_capital_eixample/', 'https://www.pisos.com//viviendas/gracia_distrito/', 'https://www.pisos.com//viviendas/horta_guinardo/', 'https://www.pisos.com//viviendas/barcelona_capital_les_corts/', 'https://www.pisos.com//viviendas/nou_barris/', 'https://www.pisos.com//viviendas/sant_andreu/', 'https://www.pisos.com//viviendas/sant_marti/', 'https://www.pisos.com//viviendas/sants_montjuic/', 'https://www.pisos.com//viviendas/sarria_sant_gervasi/', 'https://www.pisos.com//viviendas/hospitalet_de_llobregat/', 'https://www.pisos.com//viviendas/sant_ad

In [9]:
for url in urls:
    for i in range(57):
        try:
            endpoint = url + f'{i+1}'
            scrape(endpoint)
        except:
            continue

NameError: name 'dsakjdasj' is not defined

In [ ]:
urls